In [8]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [9]:
use_stack = True

train_df = pd.read_csv('input/train' + ('_stack' if use_stack else '') + '.csv')
#train_df = pd.read_csv('input/train_min.csv')
test_df = pd.read_csv('input/test'  + ('_stack' if use_stack else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
train_df.head()

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [10]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [11]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [12]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8453828656355421,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample':0.8421287738494433,
    'min_child_weight': 36.93038816860224,
    'num_threads': 4,
    'max_bin': 483
}

In [13]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=2000000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 800)
    
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_splits
    score += model.best_score['valid_1']['auc'] / n_splits

print(score)

Fold 0 started at Fri Mar 15 16:39:36 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.977917	valid_1's auc: 0.979151
[600]	training's auc: 0.979395	valid_1's auc: 0.980661
[900]	training's auc: 0.979673	valid_1's auc: 0.980861
[1200]	training's auc: 0.979833	valid_1's auc: 0.980948
[1500]	training's auc: 0.979934	valid_1's auc: 0.980971
[1800]	training's auc: 0.980017	valid_1's auc: 0.980996
[2100]	training's auc: 0.980094	valid_1's auc: 0.980987
[2400]	training's auc: 0.980154	valid_1's auc: 0.980989
Early stopping, best iteration is:
[1742]	training's auc: 0.980002	valid_1's auc: 0.981003
Fold 1 started at Fri Mar 15 16:39:58 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.978636	valid_1's auc: 0.975909
[600]	training's auc: 0.980111	valid_1's auc: 0.977575
[900]	training's auc: 0.980345	valid_1's auc: 0.977806
[1200]	training's auc: 0.98051	valid_1's auc: 0.977874
[1500]	training's auc: 0.980615	

In [14]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)